# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 21 2022 8:30AM,249111,10,CTF0011464,"Citieffe, Inc.",Released
1,Oct 21 2022 8:30AM,249114,10,8672855,"Citieffe, Inc.",Released
2,Oct 20 2022 6:20PM,249115,21,50666646,"NBTY Global, Inc.",Released
3,Oct 20 2022 5:58PM,249041,21,631087,"NBTY Global, Inc.",Released
4,Oct 20 2022 4:14PM,249110,16,ACG-2102491625,ACG North America LLC,Released
5,Oct 20 2022 4:03PM,249109,19,ADV80006880,"AdvaGen Pharma, Ltd",Released
6,Oct 20 2022 3:27PM,249106,19,ADV80006879,"AdvaGen Pharma, Ltd",Released
7,Oct 20 2022 2:49PM,249040,21,631086,"NBTY Global, Inc.",Executing
8,Oct 20 2022 2:15PM,249094,10,0086092505,ISDIN Corporation,Released
9,Oct 20 2022 2:15PM,249094,10,0086092508,ISDIN Corporation,Completed


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,249109,Released,1
15,249110,Released,1
16,249111,Released,1
17,249114,Released,1
18,249115,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249109,NaN,NaN,1.0
249110,NaN,NaN,1.0
249111,NaN,NaN,1.0
249114,NaN,NaN,1.0
249115,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249040,0.0,1.0,0.0
249041,0.0,0.0,1.0
249076,0.0,0.0,1.0
249079,0.0,0.0,1.0
249082,11.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249040,0,1,0
249041,0,0,1
249076,0,0,1
249079,0,0,1
249082,11,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249040,0,1,0
1,249041,0,0,1
2,249076,0,0,1
3,249079,0,0,1
4,249082,11,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249040,,1,
1,249041,,,1
2,249076,,,1
3,249079,,,1
4,249082,11,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 21 2022 8:30AM,249111,10,"Citieffe, Inc."
1,Oct 21 2022 8:30AM,249114,10,"Citieffe, Inc."
2,Oct 20 2022 6:20PM,249115,21,"NBTY Global, Inc."
3,Oct 20 2022 5:58PM,249041,21,"NBTY Global, Inc."
4,Oct 20 2022 4:14PM,249110,16,ACG North America LLC
5,Oct 20 2022 4:03PM,249109,19,"AdvaGen Pharma, Ltd"
6,Oct 20 2022 3:27PM,249106,19,"AdvaGen Pharma, Ltd"
7,Oct 20 2022 2:49PM,249040,21,"NBTY Global, Inc."
8,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation
19,Oct 20 2022 2:04PM,249090,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 21 2022 8:30AM,249111,10,"Citieffe, Inc.",,,1
1,Oct 21 2022 8:30AM,249114,10,"Citieffe, Inc.",,,1
2,Oct 20 2022 6:20PM,249115,21,"NBTY Global, Inc.",,,1
3,Oct 20 2022 5:58PM,249041,21,"NBTY Global, Inc.",,,1
4,Oct 20 2022 4:14PM,249110,16,ACG North America LLC,,,1
5,Oct 20 2022 4:03PM,249109,19,"AdvaGen Pharma, Ltd",,,1
6,Oct 20 2022 3:27PM,249106,19,"AdvaGen Pharma, Ltd",,,1
7,Oct 20 2022 2:49PM,249040,21,"NBTY Global, Inc.",,1,
8,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation,1,,10
9,Oct 20 2022 2:04PM,249090,10,Bio-PRF,6,1,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 8:30AM,249111,10,"Citieffe, Inc.",1,,
1,Oct 21 2022 8:30AM,249114,10,"Citieffe, Inc.",1,,
2,Oct 20 2022 6:20PM,249115,21,"NBTY Global, Inc.",1,,
3,Oct 20 2022 5:58PM,249041,21,"NBTY Global, Inc.",1,,
4,Oct 20 2022 4:14PM,249110,16,ACG North America LLC,1,,
5,Oct 20 2022 4:03PM,249109,19,"AdvaGen Pharma, Ltd",1,,
6,Oct 20 2022 3:27PM,249106,19,"AdvaGen Pharma, Ltd",1,,
7,Oct 20 2022 2:49PM,249040,21,"NBTY Global, Inc.",,1,
8,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation,10,,1
9,Oct 20 2022 2:04PM,249090,10,Bio-PRF,1,1,6


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 8:30AM,249111,10,"Citieffe, Inc.",1,,
1,Oct 21 2022 8:30AM,249114,10,"Citieffe, Inc.",1,,
2,Oct 20 2022 6:20PM,249115,21,"NBTY Global, Inc.",1,,
3,Oct 20 2022 5:58PM,249041,21,"NBTY Global, Inc.",1,,
4,Oct 20 2022 4:14PM,249110,16,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 8:30AM,249111,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Oct 21 2022 8:30AM,249114,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Oct 20 2022 6:20PM,249115,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Oct 20 2022 5:58PM,249041,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Oct 20 2022 4:14PM,249110,16,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 8:30AM,249111,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Oct 21 2022 8:30AM,249114,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Oct 20 2022 6:20PM,249115,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Oct 20 2022 5:58PM,249041,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Oct 20 2022 4:14PM,249110,16,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1245491,16.0,1.0,18.0
16,498196,2.0,0.0,0.0
19,498215,2.0,0.0,0.0
21,996284,3.0,1.0,0.0
22,498155,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1245491,16.0,1.0,18.0
1,16,498196,2.0,0.0,0.0
2,19,498215,2.0,0.0,0.0
3,21,996284,3.0,1.0,0.0
4,22,498155,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,16.0,1.0,18.0
1,16,2.0,0.0,0.0
2,19,2.0,0.0,0.0
3,21,3.0,1.0,0.0
4,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,16.0
1,16,Released,2.0
2,19,Released,2.0
3,21,Released,3.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21,22
Status,,,,,
Completed,18.0,0.0,0.0,0.0,0.0
Executing,1.0,0.0,0.0,1.0,0.0
Released,16.0,2.0,2.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21,22
0,Completed,18.0,0.0,0.0,0.0,0.0
1,Executing,1.0,0.0,0.0,1.0,0.0
2,Released,16.0,2.0,2.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21,22
0,Completed,18.0,0.0,0.0,0.0,0.0
1,Executing,1.0,0.0,0.0,1.0,0.0
2,Released,16.0,2.0,2.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()